In [1]:
import os, numpy as np
import pandas as pd

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras 
from sklearn import datasets

from sklearn.model_selection import train_test_split

from keras.utils import np_utils



/home/antoine/.local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


# Build a Keras Model

In [16]:


def create_model():
    from keras.models import Sequential
    from keras.layers.core import Dense, Activation

    model = Sequential()
    model.add(Dense(12, input_dim=13))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [17]:
boston  = datasets.load_boston()
train_X, test_X, train_y, test_y = train_test_split(boston.data, boston.target, train_size=0.8, test_size=0.2, random_state=1960)


In [18]:

from keras.wrappers.scikit_learn import KerasRegressor

clf = KerasRegressor(build_fn=create_model, epochs=12, verbose=0)

print(train_X.shape , train_y.shape)
clf.fit(train_X, train_y, verbose=0)

(404, 13) (404,)


In [19]:
print(test_X.shape)
preds = clf.predict(test_X[0,:].reshape(1,13))
print(preds)


(102, 13)
38.092304


# Generate SQL Code from the Model

In [20]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

# no luck for the web service... pickling feature of tensorflow and/or keras objects seems not to be a priority.
# there is a lot of github issues in the two projects when I search for pickle keyword!!!.

def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



def test_sql_gen(keras_regressor , metadata):
    import sklearn2sql.PyCodeGenerator as codegen
    cg1 = codegen.cAbstractCodeGenerator();
    lSQL = cg1.generateCodeWithMetadata(clf, metadata, dsn = None, dialect = "postgresql");
    return lSQL[0]


In [21]:
# commented .. see above
# pickle_data = pickle.dumps(clf)
# lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [22]:
lMetaData = {}
lMetaData['features'] = list(boston.feature_names);
lMetaData["targets"] = ['TGT']
lMetaData['primary_key'] = 'KEY'
lMetaData['table'] = 'boston'

    
lSQL = test_sql_gen(clf , lMetaData)


/usr/local/lib/python3.6/dist-packages/xgboost
cGenerationWrapperFactory::createWrapper() <class 'keras.wrappers.scikit_learn.KerasRegressor'>
BACKEND_DIALECT postgresql
CREATING_DATABASE_BACKEND_DSN_DIALECT 1.2.2 None postgresql
KERAS_REG_GENERATE_EXPRESSION_START <class 'sqlalchemy.sql.selectable.Alias'> <class 'sklearn2sql.GenericModels.Keras.cSerializedKeras'>
{'mKerasData': <keras.wrappers.scikit_learn.KerasRegressor object at 0x7faf3afcdf98>, 'mFeatureNames': ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'], 'mClasses': None, 'mMode': 'Regression'}
{'build_fn': <function create_model at 0x7faf3afb79d8>, 'sk_params': {'epochs': 12, 'verbose': 0}, 'model': <keras.models.Sequential object at 0x7faf3f2c0e10>}
ABSTRACT_GENERATE_CTE keras_input keras_input 14
GENERATING_LAYER 0 dense_5 <class 'keras.layers.core.Dense'> {'input_spec': <keras.engine.topology.InputSpec object at 0x7faf3f2c0518>, 'supports_masking': True, '_trainable_weights

In [23]:
print(lSQL)

WITH keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."CRIM" AS "CRIM", "ADS"."ZN" AS "ZN", "ADS"."INDUS" AS "INDUS", "ADS"."CHAS" AS "CHAS", "ADS"."NOX" AS "NOX", "ADS"."RM" AS "RM", "ADS"."AGE" AS "AGE", "ADS"."DIS" AS "DIS", "ADS"."RAD" AS "RAD", "ADS"."TAX" AS "TAX", "ADS"."PTRATIO" AS "PTRATIO", "ADS"."B" AS "B", "ADS"."LSTAT" AS "LSTAT" 
FROM boston AS "ADS"), 
layer_dense_5 AS 
(SELECT keras_input."KEY" AS "KEY", 0.0110807865858078 + -0.11323320120573044 * keras_input."CRIM" + -0.31711265444755554 * keras_input."ZN" + 0.46612322330474854 * keras_input."INDUS" + 0.45720869302749634 * keras_input."CHAS" + 0.19844095408916473 * keras_input."NOX" + 0.2507675588130951 * keras_input."RM" + -0.04160049557685852 * keras_input."AGE" + -0.21529483795166016 * keras_input."DIS" + 0.4905403256416321 * keras_input."RAD" + -0.06454608589410782 * keras_input."TAX" + 0.028352104127407074 * keras_input."PTRATIO" + 0.14279663562774658 * keras_input."B" + 0.4624735116958618 * keras_input."LSTAT"

# Execute the SQL Code

In [24]:
# save the dataset in a database table


import sqlalchemy as sa

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(boston.data);
lTable.columns = lMetaData['features']
lTable['TGT'] = boston.target
lTable['KEY'] = range(boston.data.shape[0])
lTable.to_sql(lMetaData['table'] , conn,   if_exists='replace', index=False)

In [25]:
sql_output = pd.read_sql(lSQL , conn);

In [26]:
sql_output.sample(12, random_state=1960)

,KEY,Estimator
230,230,38.092299
112,112,32.701267
125,125,61.067988
9,9,43.036557
213,213,23.843837
109,109,37.850608
127,127,31.470673
244,244,34.423350
406,406,18.846141
490,490,-14.846010


# Keras Prediction

In [27]:
skl_output = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(boston.data.shape[0])), columns=['KEY']);
skl_output_estimator = pd.DataFrame(clf.predict(boston.data), columns=['Estimator'])
skl_output['KEY'] = skl_output_key
skl_output['Estimator'] = skl_output_estimator
skl_output.sample(12, random_state=1960)

,KEY,Estimator
230,230,38.092297
112,112,32.701267
125,125,61.067951
9,9,43.036564
213,213,23.843824
109,109,37.850620
127,127,31.470648
244,244,34.423344
406,406,18.846121
490,490,-14.845973


# Comparing the SQL and Keras Predictions

In [28]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')
sql_skl_join['Error'] = sql_skl_join.Estimator_sql - sql_skl_join.Estimator_skl

In [29]:
sql_skl_join.head(12)

,KEY_skl,Estimator_skl,KEY_sql,Estimator_sql,Error
0,0,33.847477,0,33.847512,0.000035
1,1,48.849232,1,48.849204,-0.000028
2,2,39.348309,2,39.348297,-0.000012
3,3,36.128597,3,36.128612,0.000015
4,4,40.555729,4,40.555736,0.000007
5,5,42.288509,5,42.288516,0.000007
6,6,35.348026,6,35.348052,0.000025
7,7,49.538776,7,49.538779,0.000002
8,8,52.205700,8,52.205727,0.000028
9,9,43.036564,9,43.036557,-0.000007


In [30]:
sql_skl_join.describe()

,KEY_skl,Estimator_skl,KEY_sql,Estimator_sql,Error
count,506.000000,506.000000,506.000000,506.000000,506.000000
mean,252.500000,22.295050,252.500000,22.295056,0.000006
std,146.213884,21.583561,146.213884,21.583562,0.000016
min,0.000000,-46.944653,0.000000,-46.944655,-0.000050
25%,126.250000,9.775058,126.250000,9.775059,-0.000004
50%,252.500000,24.404417,252.500000,24.404425,0.000007
75%,378.750000,37.792643,378.750000,37.792634,0.000017
max,505.000000,65.194275,505.000000,65.194293,0.000054
